This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

## Beyond text classification: Sequence-to-sequence learning

### A machine translation example

In [1]:
text_file = "../data/spa-eng/spa.txt"
with open(text_file, encoding='utf8') as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [2]:
import random
print(random.choice(text_pairs))

('I should talk to Tom.', '[start] Debería hablar con Tom. [end]')


In [3]:
import random
# 打乱
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
# 划分为训练集、验证集和测试集
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

**Vectorizing the English and Spanish text pairs**

In [5]:
import tensorflow as tf
import string
import re
from tensorflow.keras import layers

# 为西班牙语的TextVectorization层准备一个自定义的字符串标准化函数
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

# 查看每种语言前15000个最常见的单词，并将句子长度限制为20个单词
vocab_size = 15000
sequence_length = 20

# 英语
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# 西班牙语
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
# 学习每种语言的词表
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

**Preparing datasets for the translation task**

In [6]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    # 保证输入和目标具有相同的长度
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    # 利用内存缓存来加快预处理速度
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [7]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


### Sequence-to-sequence learning with RNNs

**GRU-based encoder**

In [8]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

**GRU-based decoder and the end-to-end model**

In [9]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

**Training our recurrent sequence-to-sequence model**

In [10]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1302/1302 [==============================] - 78s 53ms/step - loss: 1.6437 - accuracy: 0.4136 - val_loss: 1.3242 - val_accuracy: 0.5011
Epoch 2/15
1302/1302 [==============================] - 67s 52ms/step - loss: 1.3153 - accuracy: 0.5256 - val_loss: 1.1559 - val_accuracy: 0.5677
Epoch 3/15
1302/1302 [==============================] - 68s 52ms/step - loss: 1.1729 - accuracy: 0.5766 - val_loss: 1.0746 - val_accuracy: 0.5997
Epoch 4/15
1302/1302 [==============================] - 68s 52ms/step - loss: 1.0822 - accuracy: 0.6085 - val_loss: 1.0434 - val_accuracy: 0.6169
Epoch 5/15
1302/1302 [==============================] - 68s 53ms/step - loss: 1.0341 - accuracy: 0.6332 - val_loss: 1.0270 - val_accuracy: 0.6273
Epoch 6/15
1302/1302 [==============================] - 69s 53ms/step - loss: 1.0029 - accuracy: 0.6518 - val_loss: 1.0229 - val_accuracy: 0.6324
Epoch 7/15
1302/1302 [==============================] - 69s 53ms/step - loss: 0.9836 - accuracy: 0.6652 - val_loss: 1.0217 -

**Translating new sentences with our RNN encoder and decoder**

In [11]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    # 种子词元
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        # 下一个词元进行采样
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
He has gone to the library.
1/1 [==============================] - 0s 24ms/step
[start] se ha ido a la biblioteca [end]
-
Tom didn't say how much money he had spent.
1/1 [==============================] - 0s 25ms/step
[start] tom no dijo cuánto dinero tenía [end]
-
These are my books and those are his books.
1/1 [==============================] - 0s 26ms/step
[start] estos son libros y mi son los libros [end]
-
He had to carry the bag.
1/1 [==============================] - 0s 24ms/step
[start] Él tuvo que la [UNK] [end]
-
You shall have my answer tomorrow.
1/1 [==============================] - 0s 25ms/step
[start] mañana tienes mi respuesta [end]
-
You must always keep your hands clean.
1/1 [==============================] - 0s 25ms/step
[start] siempre tienes que [UNK] tus manos [end]
-
Happy workers are better workers.
1/1 [==============================] - 0s 24ms/step
[start] los hombres de los hombres son demasiado [UNK] [end]
-
There's too much salt in this soup.
1/1 [=======

1/1 [==============================] - 0s 50ms/step
[start] puedo uno de estos [end]
-
The property was divided equally among the heirs.
1/1 [==============================] - 0s 35ms/step
[start] la [UNK] estaba muy [UNK] del [UNK] [end]
-
This is the third time I've heard this song today.
1/1 [==============================] - 0s 38ms/step
[start] esta es la única vez que he visto esta canción ayer [end]
-
I'll pop in tomorrow morning.
1/1 [==============================] - 0s 52ms/step
[start] me voy a la mañana mañana [end]


### Sequence-to-sequence learning with Transformer

#### The Transformer decoder

**The `TransformerDecoder`**

In [12]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        # 将输入掩码传递给输出
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config
    
    # 生成因果掩码
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)
    
    # 实现前向传播
    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        # 准备输入掩码
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            # 合并掩码
            padding_mask = tf.minimum(padding_mask, causal_mask)
        # 将因果掩码传入第一个注意力层    
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        # 将合并后的掩码传入第二个注意力层
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### Putting it all together: A Transformer for machine translation

**PositionalEmbedding layer**

In [13]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**End-to-end Transformer**

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        # 输入词元向量的尺寸
        self.embed_dim = embed_dim
        # Dense层的尺寸
        self.dense_dim = dense_dim
        # 注意力头的个数
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        # 规范化层
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        # 增加维度
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        # 实现序列化，用于保存模型
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [16]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
# 对源句子进行编码
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
# 对目标句子进行编码，并将其与编码后的源句子合并
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

**Training the sequence-to-sequence Transformer**

In [17]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 66s 48ms/step - loss: 1.6758 - accuracy: 0.4188 - val_loss: 1.3484 - val_accuracy: 0.5002
Epoch 2/30
1302/1302 [==============================] - 63s 48ms/step - loss: 1.3404 - accuracy: 0.5319 - val_loss: 1.1672 - val_accuracy: 0.5693
Epoch 3/30
1302/1302 [==============================] - 63s 48ms/step - loss: 1.1895 - accuracy: 0.5814 - val_loss: 1.0911 - val_accuracy: 0.6002
Epoch 4/30
1302/1302 [==============================] - 63s 49ms/step - loss: 1.1005 - accuracy: 0.6130 - val_loss: 1.0483 - val_accuracy: 0.6203
Epoch 5/30
1302/1302 [==============================] - 64s 49ms/step - loss: 1.0503 - accuracy: 0.6357 - val_loss: 1.0199 - val_accuracy: 0.6335
Epoch 6/30
1302/1302 [==============================] - 63s 48ms/step - loss: 1.0180 - accuracy: 0.6520 - val_loss: 1.0066 - val_accuracy: 0.6412
Epoch 7/30
1302/1302 [==============================] - 63s 49ms/step - loss: 0.9943 - accuracy: 0.6653 - val_loss: 1.0024 -

**Translating new sentences with our Transformer model**

In [18]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
The tigers escaped from the zoo.
[start] los [UNK] de que los [UNK] de el teatro [end]
-
Washing your hands regularly is a good way to protect yourself from diseases.
[start] se me [UNK] tus manos una buena [UNK] para [UNK] de a que te [UNK] [end]
-
I wasn't prepared.
[start] no estaba lista [end]
-
Tom did it.
[start] tom lo hizo [end]
-
Do you want to go for a swim?
[start] quieres ir a nadar [end]
-
I was asked to wait here.
[start] me [UNK] [end]
-
I know someone who may be interested in your project.
[start] sé alguien que tom pueda estar en tu [UNK] [end]
-
From there, one could see perfectly.
[start] desde ahí podía ver la la la la la la la [UNK] [end]
-
I'll text you when I'm done.
[start] te voy a [UNK] de repente [end]
-
Tom avoided Mary.
[start] tom le dio a mary [end]
-
Tom doesn't know what Mary has to do.
[start] tom no sabe qué tiene mary que hacer [end]
-
She worked for a rich man.
[start] ella funcionó por un hombre rico [end]
-
She's beautiful.
[start] es hermosa [e

## Summary